In [ ]:
import numpy as np
from scipy.integrate import solve_ivp
from time import time
from numba import njit
tol = 1e-8
nn = 301
x = 0.0
y = 0.0
betaBX = 200.0
betaFX = 10.5
kappaX = 1.2
kappaY = 0.9
theta = 1.0
alpha = 10.0
betaBY = 400.0
betaFY = 10.0
deltaX = 1.0
deltaY = 0.05
par = betaFX
IC = state = np.array([x,y])

stateDim, parDim = len(state), 1

@njit
def F(t,state,par):  
    x,y = state
    betaFX = par  
    tempX = x * kappaX / theta
    tempY = x * kappaY / theta
    dxdt = betaFX + (betaBX - betaFX) * (tempX**3) / (1 + tempX + tempX**2 + tempX**3) - deltaX*x - alpha*x*y
    dydt = betaFY + (betaBY - betaFY) * (tempY**3) / (1 + tempY + tempY**2 + tempY**3) - deltaY*y - alpha*x*y    
    return np.array([dxdt,dydt])
        
@njit
def J(t,state,par):
    x,y = state
    betaFX = par
    return np.array([[(-kappaX**4*x**3*(betaBX - betaFX)*(3*kappaX**2*x**2 + 2*kappaX*theta*x + theta**2) + 3*kappaX**3*x**2*(betaBX - betaFX)*(kappaX**3*x**3 + kappaX**2*theta*x**2 + kappaX*theta**2*x + theta**3) - (alpha*y + deltaX)*(kappaX**3*x**3 + kappaX**2*theta*x**2 + kappaX*theta**2*x + theta**3)**2)/(kappaX**3*x**3 + kappaX**2*theta*x**2 + kappaX*theta**2*x + theta**3)**2, -alpha*x], [(-alpha*y*(kappaY**3*x**3 + kappaY**2*theta*x**2 + kappaY*theta**2*x + theta**3)**2 - kappaY**4*x**3*(betaBY - betaFY)*(3*kappaY**2*x**2 + 2*kappaY*theta*x + theta**2) + 3*kappaY**3*x**2*(betaBY - betaFY)*(kappaY**3*x**3 + kappaY**2*theta*x**2 + kappaY*theta**2*x + theta**3))/(kappaY**3*x**3 + kappaY**2*theta*x**2 + kappaY*theta**2*x + theta**3)**2, -alpha*x - deltaY]])

@njit
def dFdtheta_constant(t,state,par):
    x,y = state
    betaFX = par
    tempX = x * kappaX / theta    
    return np.array([[1-(tempX**3) / (1 + tempX + tempX**2 + tempX**3)],[0.0]])
    
def dJdx(t,state,par): 
    x,y = state
    betaFX = par
    dflatJdx = np.array([[kappaX**3*x**3*(betaBX - betaFX)*(-6*kappaX**3*x/theta**3 - 2*kappaX**2/theta**2)/(theta**3*(kappaX**3*x**3/theta**3 + kappaX**2*x**2/theta**2 + kappaX*x/theta + 1)**2) + kappaX**3*x**3*(betaBX - betaFX)*(-6*kappaX**3*x**2/theta**3 - 4*kappaX**2*x/theta**2 - 2*kappaX/theta)*(-3*kappaX**3*x**2/theta**3 - 2*kappaX**2*x/theta**2 - kappaX/theta)/(theta**3*(kappaX**3*x**3/theta**3 + kappaX**2*x**2/theta**2 + kappaX*x/theta + 1)**3) + 6*kappaX**3*x**2*(betaBX - betaFX)*(-3*kappaX**3*x**2/theta**3 - 2*kappaX**2*x/theta**2 - kappaX/theta)/(theta**3*(kappaX**3*x**3/theta**3 + kappaX**2*x**2/theta**2 + kappaX*x/theta + 1)**2) + 6*kappaX**3*x*(betaBX - betaFX)/(theta**3*(kappaX**3*x**3/theta**3 + kappaX**2*x**2/theta**2 + kappaX*x/theta + 1)), -alpha], [-alpha, 0], [kappaY**3*x**3*(betaBY - betaFY)*(-6*kappaY**3*x/theta**3 - 2*kappaY**2/theta**2)/(theta**3*(kappaY**3*x**3/theta**3 + kappaY**2*x**2/theta**2 + kappaY*x/theta + 1)**2) + kappaY**3*x**3*(betaBY - betaFY)*(-6*kappaY**3*x**2/theta**3 - 4*kappaY**2*x/theta**2 - 2*kappaY/theta)*(-3*kappaY**3*x**2/theta**3 - 2*kappaY**2*x/theta**2 - kappaY/theta)/(theta**3*(kappaY**3*x**3/theta**3 + kappaY**2*x**2/theta**2 + kappaY*x/theta + 1)**3) + 6*kappaY**3*x**2*(betaBY - betaFY)*(-3*kappaY**3*x**2/theta**3 - 2*kappaY**2*x/theta**2 - kappaY/theta)/(theta**3*(kappaY**3*x**3/theta**3 + kappaY**2*x**2/theta**2 + kappaY*x/theta + 1)**2) + 6*kappaY**3*x*(betaBY - betaFY)/(theta**3*(kappaY**3*x**3/theta**3 + kappaY**2*x**2/theta**2 + kappaY*x/theta + 1)), -alpha], [-alpha, 0.0]])
    dJdx = np.reshape(dflatJdx, (stateDim, stateDim, stateDim))
    return dJdx

def d2Fdthetadx_constant(t,state,par):
    x,y = state
    betaFX = par
    return np.array([[-kappaX**3*x**3*(-3*kappaX**3*x**2/theta**3 - 2*kappaX**2*x/theta**2 - kappaX/theta)/(theta**3*(kappaX**3*x**3/theta**3 + kappaX**2*x**2/theta**2 + kappaX*x/theta + 1)**2) - 3*kappaX**3*x**2/(theta**3*(kappaX**3*x**3/theta**3 + kappaX**2*x**2/theta**2 + kappaX*x/theta + 1)), 0.0], [0.0, 0.0]])
    
def jointBackward(t,jointState,par):
    xx = jointState[:stateDim]
    adj = jointState[stateDim:2*stateDim]
    dx = F(t,xx,par).reshape((stateDim,))
    dadj = -(adj.reshape([1,stateDim]).dot(J(t,xx,par))).reshape((stateDim,))
    dsens = -(adj.reshape([1,stateDim]).dot(dFdtheta_constant(t,xx,par))).reshape((parDim,))
    return np.hstack((dx, dadj, dsens))

def jointJacobian(t,jointState,par):
    jointJ = np.zeros((2*stateDim + parDim, 2*stateDim + parDim))
    xx = jointState[:stateDim]
    adj = jointState[stateDim:2*stateDim]
    jointJ[:stateDim, :stateDim] = J(t,xx,par)[:,:]
    jointJ[stateDim:2*stateDim, stateDim:2*stateDim] = - J(t,xx,par).T[:,:]
    dJdxval = dJdx(t,xx,par)
    jointJ[stateDim:2*stateDim, :stateDim] = - np.tensordot(adj.reshape([1,stateDim]), dJdxval, axes=(1,0))
    jointJ[2*stateDim:, stateDim:2*stateDim] = -dFdtheta_constant(t,xx,par).T
    d2FdthetadxVal = d2Fdthetadx_constant(t,xx,par)
    jointJ[2*stateDim:, :stateDim] = -np.tensordot(adj.reshape([1,stateDim]), d2FdthetadxVal, axes=(1,0))
    return jointJ
        
def forwardSimulation(par):
    state = np.copy(IC)
    sol_buffer = solve_ivp(fun=lambda t,z: F(t,z,par), jac=lambda t,z:J(t,z,par), t_span=(tSpan[0],tSpan[-1]),y0=state, t_eval=tSpan, method='LSODA', rtol=tol,atol=tol)
    return sol_buffer.t, sol_buffer.y

def adjointSensitivities(par):
    t,fullState = forwardSimulation(par)
    jointState = np.zeros(2*stateDim+parDim)
    jointState[:stateDim] = fullState[:,-1]
    jointState[stateDim:2*stateDim] = np.array([1,0])
    sol_buffer = solve_ivp(fun=lambda t,z: jointBackward(t,z,par), jac=lambda t,z: jointJacobian(t,z,par), t_span=(tSpan[[-1,0]]),y0=jointState, t_eval=tSpan[::-1][[0,-1]], method='LSODA',rtol=tol,atol=tol)
    jointState[:] = sol_buffer.y[:,1]
    return jointState[2*stateDim]

temp1 = np.zeros(nn)
temp2 = np.zeros(nn)
count = 0
tf_arr = np.linspace(0,20,nn)
for ii in range(1,nn):
    tf = tf_arr[ii]
    tSpan = np.linspace(0,tf,1001)    
    count += 1
    print(count)
    for jj in range(2):
        tic = time()
        ww = adjointSensitivities(par)
        temp1[count] = time()-tic
    temp2[count] = ww        

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
 lsoda--  warning..internal t (=r1) and h (=r2) are  
       such that in the machine, t + h = t on the next step  
       (h = step size). solver will continue anyway  
      in above,  r1 =  0.3007089551973D-02   r2 = -0.0000000000000D+00
 lsoda--  warning..internal t (=r1) and h (=r2) are  
       such that in the machine, t + h = t on the next step  
       (h = step size). solver will continue anyway  
      in above,  r1 =  0.3007089551973D-02   r2 = -0.0000000000000D+00
 lsoda--  warning..internal t (=r1) and h (=r2) are  
       such that in the machine, t + h = t on the next step  
       (h = step size). solver will continue anyway  
      in above,  r1 =  0.3007089551973D-02   r2 = -0.0000000000000D+00
 lsoda--  warning..int

Traceback (most recent call last):
capi_return is NULL
Call-back cb_f_in_lsoda__user__routines failed.
Fatal Python error: F2PySwapThreadLocalCallbackPtr: F2PySwapThreadLocalCallbackPtr: PyLong_AsVoidPtr failed
Python runtime state: initialized
  File "/Users/amallela/Library/Python/3.9/lib/python/site-packages/scipy/integrate/_ivp/base.py", line 152, in fun
    def fun(t, y):
KeyboardInterrupt


In [17]:
np.save('adjoint1.npy',temp1)
np.save('adjoint2.npy',temp2)